In [ ]:
#input must be options chain day1 - end

In [1]:
import os,h5py
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from IPython.display import clear_output
import joblib
import random
import hashlib
from src.wgangp.utils import Scaler

In [94]:
CSV_PATH     = './data/OptionsEOD.csv/'
#create @ part 1
PARQUET_PATH = './data/OptionsEOD.parquet'
#create @ part 2
PARQUET_STG_PATH = './data/OptionsEOD_STG.parquet'
SCALER_COL  = [ 'UNDERLYING_LAST','STRIKE','STRIKE_DISTANCE','INTRINSIC_VALUE','DTE','C_VEGA','P_VEGA',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK', 'P_VOLUME' ]
#create @ part 3
OTHER_COL = ['OPTIONS_ID','QUOTE_UNIXTIME', 'EXPIRE_UNIX']

QUOTE_COL = ['C_BID',	'C_ASK',  'P_BID',	'P_ASK']
VEGA_COL =  ["C_VEGA","P_VEGA"] 
VOLUME_COL =  ["TOTAL_VOLUME","C_VOLUME","P_VOLUME"] 

SCALER_QUOTE_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v in QUOTE_COL]
SCALER_VEGA_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v in VEGA_COL]
SCALER_VOLUME_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v in VOLUME_COL]
SCALER_OTHER_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v not in QUOTE_COL+VEGA_COL+VOLUME_COL ]


UNIQUE_KEYS = ['SYMBOL','EXPIRE_DATE']
H5_PATH = './data/OptTrainRenforcement/'
START = True#True#'2010-09'#True


max_option_len = 16

In [73]:
#==== Create options_qoute
options_qoute = {}

In [ ]:
#example
EOD_CSV = pd.read_csv(CSV_PATH+"qqq/qqq_eod_201201.txt", engine='pyarrow')
EOD_CSV.head()

In [ ]:
==================================Create OptionsEOD.parquet part 1========================================

In [31]:
#Part I
#TransformData : 
#-each partition from EXPIRE_DATE 
#-csv too parquet
#-col. rename 
def TransformDataI():
    #scaler = MinMaxScaler()
    scaler = MinMaxScaler()
    
    schema = None
    pqwriter = None
    for d in os.listdir(CSV_PATH):
        for f in os.listdir(CSV_PATH+f"{d}/"):
            if f.endswith(".txt"):
                ## load
                print( f"[LOAD] : {CSV_PATH}{d}/{f}        ",end='\r')
                EOD_CSV = pd.read_csv(CSV_PATH+f"{d}/"+f, engine='pyarrow')
                    
                ## rename col.
                for c in EOD_CSV.columns:
                    EOD_CSV = EOD_CSV.rename( columns={ c:c.strip().replace(']','').replace('[','') } )
                
                ## add symbol 
                EOD_CSV['SYMBOL'] = d.upper()
                ## add INTRINSIC_VALUE
                EOD_CSV['INTRINSIC_VALUE'] = EOD_CSV['UNDERLYING_LAST'] - EOD_CSV['STRIKE']
                
                ## fillnafillna
                EOD_CSV['P_VOLUME'] = EOD_CSV['P_VOLUME'].fillna(0)
                EOD_CSV['C_VOLUME'] = EOD_CSV['C_VOLUME'].fillna(0)
                EOD_CSV.dropna(subset=[SCALER_COL])
            
                
                # date columns convert to datetime
                for c in ["QUOTE_READTIME","QUOTE_DATE","EXPIRE_DATE"]:
                    EOD_CSV[c] = pd.to_datetime(EOD_CSV[c])
                
                #clean float data
                for c in ['INTRINSIC_VALUE','C_DELTA','C_GAMMA','C_VEGA','C_THETA','C_RHO','C_IV','C_VOLUME','C_LAST','C_BID','C_ASK','STRIKE','P_BID','P_ASK','P_LAST','P_DELTA','P_GAMMA','P_VEGA','P_THETA','P_RHO','P_IV','P_VOLUME','STRIKE_DISTANCE','STRIKE_DISTANCE_PCT']:
                    if EOD_CSV[c].dtype not in ( 'float32','float64'):
                        EOD_CSV[c] = EOD_CSV[c].apply(lambda x: x.strip())
                        EOD_CSV[c] = EOD_CSV[c].replace('', np.nan).fillna(np.nan)
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                    if EOD_CSV[c].dtype == 'float32':
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                        
                # REMAIN_DAYS(int) =>  use DTE col.
                #partition with QUOTE_DATE
                EOD_CSV['PartitionDate'] = EOD_CSV['QUOTE_DATE'].dt.strftime('%Y-%m')
                EOD_CSV.sort_values(['QUOTE_DATE','EXPIRE_DATE','SYMBOL','STRIKE'],ascending =False ) 

                #scaler(Normalization_
                #scaler.partial_fit(EOD_CSV[SCALER_COL])

                # save
                if os.path.exists(PARQUET_PATH):
                  EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
                else:
                  EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )
                    
    # joblib.dump(scaler, SCALER_PATH )
    # if pqwriter:
    #     pqwriter.close()
    # print( f"[DONE]                                                       ",end='\r')



In [ ]:
##-RunCleanData
#TransformDataI()

In [ ]:
==================================Create OptTrainData STG part 2========================================

In [141]:
##### Part II 
from tqdm import tqdm
#START = '2012-02'
#TransformData : 
# - read each partitions 
# - Normalization if not have scaler.gz file
# ###===== load Scaler
# SCALER = Scaler(
#     [v for v in SCALER_COL if v not in QUOTE_COL+VEGA_COL+VOLUME_COL ] + ['QUOTE','VEGA','VOLUME']
#     , create=True
#     , path="./data/scaler/")

if START == True:
    pass
###===== local functions
def strikeZero(df,v,num_rm):   
    # First, filter based on QUOTE_DATE, SYMBOL, and EXPIRE_DATE
    filtered_arr = df[(df['QUOTE_DATE'] == v['QUOTE_DATE']) &
                     (df['SYMBOL'] == v['SYMBOL']) &
                     (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])]['STRIKE'].values
    
    if num_rm != 0 :
        df.loc[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
                & (df['SYMBOL'] == v['SYMBOL']) 
                & (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])
                & (  (df['STRIKE'].isin(filtered_arr[:num_rm])  )
                   | (df['STRIKE'].isin(filtered_arr[-num_rm:])   )
                  ) 
            , SCALER_COL    
        ] = 0
    
def hash_str(S):
    return int(hashlib.md5(S.encode('utf-8')).hexdigest(),16)
    
#========================================= main model =========================================
keys = None#df[unique_keys].sort_values(by=unique_keys).drop_duplicates()

#==== Get PartitionDate
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
#PartitionDate = ['2011-12','2022-05'] # debug
debug = None
for i,partdate in  enumerate(tqdm(PartitionDate[:1]) ) :  
    START = True if START == partdate else START
    if START == True :
        df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                     , filters=[('PartitionDate', '=', partdate)]
                                    )
        df['PartitionDate'] = df['PartitionDate'].astype('object') 
        #add col options_id
        df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
        df['C_VOLUME'] = df['C_VOLUME'].fillna(0)
        df[df == 0] = 0
        df = df.dropna(subset=[c for c in SCALER_COL if c not in ('ID','QUOTE_DATE', 'EXPIRE_DATE', 'TOTAL_VOLUME') ])
        ####################################################
        keys_item = df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates()[:]
        
        #loop each UNIQUE_KEYS keys 
        for j,v in keys_item.iterrows():
            msg = {}
            msg['qd_ate'] = v['QUOTE_DATE']
            msg['symbo'] = v['SYMBOL']
            msg['exdate'] = v['EXPIRE_DATE']
            
            df_Item=df[(df['SYMBOL'] == v['SYMBOL']) & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) ]
            keys_quote = df_Item[['QUOTE_DATE']].sort_values(by=['QUOTE_DATE']).drop_duplicates()[:]
            DATA  = np.empty((0,) + ( max_option_len,len(SCALER_COL)+len(OTHER_COL) ) ) #init 
            for k,qv in keys_quote.iterrows():
                df_filter =df_Item[(df_Item['QUOTE_DATE'] == qv['QUOTE_DATE']) ]
                df_filter['OPTIONS_ID'] = int( f'{i}{j}{k}' )
                #df_filter['OPTIONS_ID'] = df_filter['OPTIONS_ID'].astype('int') 
                # df_filter['EXPIRE_DATE'] = df['EXPIRE_DATE'].astype('S10') 
                # df_filter['OPTIONS_ID'] = df['OPTIONS_ID'].astype('S10') 
                msg['start_rows'] = len(df_filter) 

                #break loops
                if len(df_filter) < 5 : 
                    print(f"""{msg['qd_ate']}|{msg['symbo']}|{msg['exdate']} : {msg['start_rows']}->break (less rows)""")
                    break 

                qoute = "".join(v[ ['SYMBOL','EXPIRE_DATE'] ].apply(str).values)
                #add new qoute
                if qoute not in [*options_qoute.keys()]:
                    options_qoute[qoute] = {}
                    options_qoute[qoute]['start_price'] = df_filter['UNDERLYING_LAST'].values[0]
                    options_qoute[qoute]['strike'] = df_filter[ df_filter['INTRINSIC_VALUE'].abs().isin(df_filter['INTRINSIC_VALUE'].abs().sort_values()[:max_option_len]) ]['STRIKE'].values
                    options_qoute[qoute]['exp'] = df_filter['EXPIRE_DATE'].values[0]
                    #check diff UNDERLYING_LAST
                    if df_filter['UNDERLYING_LAST'].values[0] != round(np.average(df_filter['UNDERLYING_LAST']),4):
                        print('[ERROR] : set UNDERLYING_LAST ',qoute )
                #rm index max : max_option_len
                #may be initstrike out of scope
                rm_strike_index = df_filter[ ~df_filter['STRIKE'].isin(options_qoute[qoute]['strike']) ].index
                df_filter = df_filter.drop(rm_strike_index)
    
                #break loops
                if len(df_filter) < 5 :  
                    print(f"""{msg['qd_ate']}|{msg['symbo']}|{msg['exdate']} : {msg['start_rows']}->break (rm_strike_index)""")
                    break
                    
                # Generate zero strike 
                # Generate a random float between 0.01 and 1
                
                # dte = df_filter[ df_filter['DTE'] != 0]['DTE'].mean()
                # df_filter['DTE'] = dte
                
                strikeZero(df_filter,v,3 )
                
                # #++++++++++++++++++++++++++++++++++++++++++++++
                # print( "A")
                # print( len(df_filter) )
                # print( nom_rm_rows )
                # print( df_filter['STRIKE_DISTANCE'] )
                # break
                # #++++++++++++++++++++++++++++++++++++++++++++++
                
                #====filed rows
                z = 0
                while len(df_filter) != max_option_len :
                    z += 1
                    if z > 50: raise "error"
                    Even= -1
                    if (len(df_filter) % 2) == 0:
                        Even = 0
                    if len(df_filter) < max_option_len :
                        zero_row = pd.DataFrame([[0]*len(df_filter.columns)], columns=df_filter.columns)
                        if Even==0 :
                            df_filter = pd.concat([df_filter, zero_row], ignore_index=True)
                        else:
                            df_filter = pd.concat([zero_row, df_filter], ignore_index=True)
                    elif len(df_filter) > max_option_len :
                         df_filter = df_filter.drop(df_filter.index[Even])
                
                #add cal columns.      
                msg['end_rows']= len(df_filter)
                if len(df_filter) == max_option_len and np.sum( df_filter[OTHER_COL+SCALER_COL].isna().values ) == 0 :
                    DATA = np.vstack((DATA ,[df_filter[OTHER_COL+SCALER_COL]]))   
                    
                #if len(DATA) :
                    # #========================== partial_fit ================================#
                    # for col_i in SCALER_OTHER_COL_INDEX:
                    #     SCALER()[SCALER_COL[col_i]].partial_fit( DATA.reshape(-1, len(SCALER_COL))[:,[col_i]] )
                    # SCALER()['QUOTE'].partial_fit( DATA.reshape(-1, len(SCALER_COL))[:,SCALER_QUOTE_COL_INDEX].reshape(-1, 1) )
                    # SCALER()['VEGA'].partial_fit( DATA.reshape(-1, len(SCALER_COL))[:,SCALER_VEGA_COL_INDEX].reshape(-1, 1) )
                    # SCALER()['VOLUME'].partial_fit( DATA.reshape(-1, len(SCALER_COL))[:,SCALER_VOLUME_COL_INDEX].reshape(-1, 1) )
                    # SCALER.save()
                    
            # #=== clear expire options_qoute
            # for qi in list(options_qoute.keys()):
            #     if options_qoute[qi]['exp'] < quote:
            #         options_qoute.pop(qi)
            
            #========================== SAVE DATA End for k============================#
            #=== save H5
            if not os.path.exists(H5_PATH):
                os.makedirs(H5_PATH)
            with h5py.File(H5_PATH+f"{partdate}.h5", 'w') as f:
                dset = f.create_dataset(f'{i}{j}{k}', data=DATA, chunks=True , compression='gzip')
            # #=== save parquet
            # if os.path.exists(PARQUET_STG_PATH):
            #   df.to_parquet(PARQUET_STG_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
            # else:
            #   df.to_parquet(PARQUET_STG_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )   
        
            debug = f"[Last Processing] {partdate}, {round(((i+1)/len(PartitionDate))*100,2)}%       "





100%|██████████| 1/1 [02:21<00:00, 141.81s/it]


In [142]:
#df_filter[OTHER_COL+SCALER_COL].dtypes

In [ ]:
debug

In [ ]:
=========================================== TEST ==============================================

In [143]:
import os
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)
print(PartitionDate)

['2012-01']


In [159]:
for partdate in PartitionDate[:1] :
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        for k in [*f.keys()]:
            DATA = f[k][:]

In [161]:
df = pd.DataFrame(DATA[0])
df.columns = OTHER_COL+SCALER_COL
df 

,OPTIONS_ID,QUOTE_UNIXTIME,EXPIRE_UNIX,UNDERLYING_LAST,STRIKE,STRIKE_DISTANCE,INTRINSIC_VALUE,DTE,C_VEGA,P_VEGA,C_BID,C_ASK,C_VOLUME,P_BID,P_ASK,P_VOLUME
0,3.553812e+09,1.325624e+09,1.419023e+09,127.54,110.0,17.5,17.54,1081.0,0.73915,0.75412,28.82,29.35,0.0,16.67,17.00,0.0
1,3.553812e+09,1.325624e+09,1.419023e+09,127.54,105.0,22.5,22.54,1081.0,0.69599,0.72060,32.02,32.55,0.0,14.88,15.26,0.0
2,3.553812e+09,1.325624e+09,1.419023e+09,127.54,100.0,27.5,27.54,1081.0,0.64692,0.68197,35.37,35.89,12.0,13.22,13.58,1.0
3,3.553812e+09,1.325624e+09,1.419023e+09,127.54,90.0,37.5,37.54,1081.0,0.53540,0.59491,42.54,43.06,0.0,10.26,10.46,27.0
4,3.553812e+09,1.325624e+09,1.419023e+09,127.54,125.0,2.5,2.54,1081.0,0.82656,0.82490,20.31,20.76,0.0,22.98,23.26,134.0
5,3.553812e+09,1.325624e+09,1.419023e+09,127.54,120.0,7.5,7.54,1081.0,0.80464,0.80796,23.01,23.39,27.0,20.73,21.00,3.0
6,3.553812e+09,1.325624e+09,1.419023e+09,127.54,115.0,12.5,12.54,1081.0,0.77589,0.78381,25.88,26.41,39.0,18.64,18.90,0.0
7,3.553812e+09,1.325624e+09,1.419023e+09,127.54,155.0,27.5,-27.46,1081.0,0.76305,0.72844,7.96,8.28,0.0,40.12,40.40,0.0
8,3.553812e+09,1.325624e+09,1.419023e+09,127.54,135.0,7.5,-7.46,1081.0,0.84244,0.83357,15.45,15.82,0.0,27.99,28.34,0.0
9,3.553812e+09,1.325624e+09,1.419023e+09,127.54,140.0,12.5,-12.46,1081.0,0.83678,0.82387,13.24,13.60,200.0,30.76,31.14,0.0


In [162]:
df = pd.DataFrame(DATA[1])
df.columns = OTHER_COL+SCALER_COL
df 

,OPTIONS_ID,QUOTE_UNIXTIME,EXPIRE_UNIX,UNDERLYING_LAST,STRIKE,STRIKE_DISTANCE,INTRINSIC_VALUE,DTE,C_VEGA,P_VEGA,C_BID,C_ASK,C_VOLUME,P_BID,P_ASK,P_VOLUME
0,3.553814e+09,1.325711e+09,1.419023e+09,127.68,135.0,7.3,-7.32,1080.0,0.84204,0.83207,15.19,15.47,0.0,27.74,28.04,0.0
1,3.553814e+09,1.325711e+09,1.419023e+09,127.68,105.0,22.7,22.68,1080.0,0.69331,0.71940,31.80,32.30,0.0,14.59,14.96,0.0
2,3.553814e+09,1.325711e+09,1.419023e+09,127.68,100.0,27.7,27.68,1080.0,0.64425,0.68035,35.18,35.73,3.0,12.97,13.26,1.0
3,3.553814e+09,1.325711e+09,1.419023e+09,127.68,95.0,32.7,32.68,1080.0,0.58870,0.63794,38.69,39.21,82.0,11.43,11.67,0.0
4,3.553814e+09,1.325711e+09,1.419023e+09,127.68,125.0,2.7,2.68,1080.0,0.82541,0.82399,20.14,20.43,0.0,22.70,23.06,11.0
5,3.553814e+09,1.325711e+09,1.419023e+09,127.68,120.0,7.7,7.68,1080.0,0.80442,0.80773,22.79,23.18,2.0,20.48,20.77,3.0
6,3.553814e+09,1.325711e+09,1.419023e+09,127.68,110.0,17.7,17.68,1080.0,0.73723,0.75420,28.55,29.06,0.0,16.46,16.73,0.0
7,3.553814e+09,1.325711e+09,1.419023e+09,127.68,90.0,37.7,37.68,1080.0,0.52982,0.59349,42.36,42.93,0.0,10.03,10.28,27.0
8,3.553814e+09,1.325711e+09,1.419023e+09,127.68,160.0,32.3,-32.32,1080.0,0.71439,0.66459,6.36,6.58,0.0,43.27,43.93,0.0
9,3.553814e+09,1.325711e+09,1.419023e+09,127.68,165.0,37.3,-37.32,1080.0,0.66324,0.59348,5.11,5.35,74.0,46.97,47.63,0.0


In [120]:
data_shape

(840, 16, 13)

In [122]:
df = pd.DataFrame(DATA[0])
df.columns = OTHER_COL+SCALER_COL
df

,OPTIONS_ID,QUOTE_UNIXTIME,EXPIRE_UNIX,UNDERLYING_LAST,STRIKE,STRIKE_DISTANCE,INTRINSIC_VALUE,DTE,C_VEGA,P_VEGA,C_BID,C_ASK,C_VOLUME,P_BID,P_ASK,P_VOLUME
0,9320.0,1.325624e+09,1.325884e+09,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00,0.00,0.0,0.00,0.00,0.0
1,9320.0,1.325624e+09,1.325884e+09,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00,0.00,0.0,0.00,0.00,0.0
2,9320.0,1.325624e+09,1.325884e+09,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00,0.00,0.0,0.00,0.00,0.0
3,9320.0,1.325624e+09,1.325884e+09,56.9,60.0,3.1,-3.1,3.0,0.00218,0.00629,0.00,0.01,20.0,3.09,3.16,0.0
4,9320.0,1.325624e+09,1.325884e+09,56.9,59.0,2.1,-2.1,3.0,0.00203,0.00692,0.00,0.01,349.0,2.10,2.15,630.0
5,9320.0,1.325624e+09,1.325884e+09,56.9,58.0,1.1,-1.1,3.0,0.01168,0.01429,0.05,0.06,3660.0,1.16,1.19,4295.0
6,9320.0,1.325624e+09,1.325884e+09,56.9,57.0,0.1,-0.1,3.0,0.02246,0.02324,0.33,0.35,9649.0,0.45,0.46,8589.0
7,9320.0,1.325624e+09,1.325884e+09,56.9,56.0,0.9,0.9,3.0,0.01472,0.01576,1.00,1.03,5008.0,0.12,0.14,14984.0
8,9320.0,1.325624e+09,1.325884e+09,56.9,55.0,1.9,1.9,3.0,0.00000,0.00707,1.88,1.93,501.0,0.03,0.04,2714.0
9,9320.0,1.325624e+09,1.325884e+09,56.9,54.0,2.9,2.9,3.0,0.00000,0.00352,2.85,2.92,10.0,0.01,0.02,60.0


In [124]:
df = pd.DataFrame(DATA[1])
df.columns = OTHER_COL+SCALER_COL
df

,OPTIONS_ID,QUOTE_UNIXTIME,EXPIRE_UNIX,UNDERLYING_LAST,STRIKE,STRIKE_DISTANCE,INTRINSIC_VALUE,DTE,C_VEGA,P_VEGA,C_BID,C_ASK,C_VOLUME,P_BID,P_ASK,P_VOLUME
0,203513.0,1.325624e+09,1.327093e+09,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00,0.00,0.0,0.00,0.00,0.0
1,203513.0,1.325624e+09,1.327093e+09,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00,0.00,0.0,0.00,0.00,0.0
2,203513.0,1.325624e+09,1.327093e+09,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00,0.00,0.0,0.00,0.00,0.0
3,203513.0,1.325624e+09,1.327093e+09,56.9,57.0,0.1,-0.1,17.0,0.04990,0.05018,0.89,0.90,16276.0,1.01,1.02,11413.0
4,203513.0,1.325624e+09,1.327093e+09,56.9,58.0,1.1,-1.1,17.0,0.04482,0.04640,0.43,0.44,9032.0,1.56,1.57,1573.0
5,203513.0,1.325624e+09,1.327093e+09,56.9,59.0,2.1,-2.1,17.0,0.03180,0.03504,0.17,0.18,7449.0,2.29,2.31,4459.0
6,203513.0,1.325624e+09,1.327093e+09,56.9,60.0,3.1,-3.1,17.0,0.01803,0.02425,0.06,0.07,4631.0,3.16,3.23,383.0
7,203513.0,1.325624e+09,1.327093e+09,56.9,61.0,4.1,-4.1,17.0,0.00947,0.01801,0.02,0.03,2564.0,4.13,4.19,550.0
8,203513.0,1.325624e+09,1.327093e+09,56.9,62.0,5.1,-5.1,17.0,0.00597,0.01433,0.01,0.02,118.0,5.10,5.18,12.0
9,203513.0,1.325624e+09,1.327093e+09,56.9,63.0,6.1,-6.1,17.0,0.00232,0.00969,0.00,0.01,16.0,6.01,6.21,0.0


In [125]:
df = pd.DataFrame(DATA[2])
df.columns = OTHER_COL+SCALER_COL
df

,OPTIONS_ID,QUOTE_UNIXTIME,EXPIRE_UNIX,UNDERLYING_LAST,STRIKE,STRIKE_DISTANCE,INTRINSIC_VALUE,DTE,C_VEGA,P_VEGA,C_BID,C_ASK,C_VOLUME,P_BID,P_ASK,P_VOLUME
0,732712.0,1.325624e+09,1.329512e+09,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00,0.00,0.0,0.00,0.00,0.0
1,732712.0,1.325624e+09,1.329512e+09,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00,0.00,0.0,0.00,0.00,0.0
2,732712.0,1.325624e+09,1.329512e+09,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00,0.00,0.0,0.00,0.00,0.0
3,732712.0,1.325624e+09,1.329512e+09,56.9,56.0,0.9,0.9,45.0,0.07620,0.07681,2.27,2.28,936.0,1.39,1.41,8785.0
4,732712.0,1.325624e+09,1.329512e+09,56.9,57.0,0.1,-0.1,45.0,0.08009,0.08029,1.65,1.67,2997.0,1.77,1.80,716.0
5,732712.0,1.325624e+09,1.329512e+09,56.9,58.0,1.1,-1.1,45.0,0.07868,0.07851,1.12,1.15,3522.0,2.25,2.28,718.0
6,732712.0,1.325624e+09,1.329512e+09,56.9,59.0,2.1,-2.1,45.0,0.07223,0.07353,0.71,0.74,627.0,2.83,2.87,242.0
7,732712.0,1.325624e+09,1.329512e+09,56.9,60.0,3.1,-3.1,45.0,0.06010,0.06410,0.42,0.44,2881.0,3.52,3.59,209.0
8,732712.0,1.325624e+09,1.329512e+09,56.9,61.0,4.1,-4.1,45.0,0.04470,0.05116,0.22,0.24,590.0,4.30,4.39,8.0
9,732712.0,1.325624e+09,1.329512e+09,56.9,62.0,5.1,-5.1,45.0,0.03036,0.03927,0.11,0.12,423.0,5.19,5.28,51.0


In [ ]:
df = pd.DataFrame(DATA[3])
df.columns = SCALER_COL
df